In [6]:
import requests
import pandas as pd

In [7]:
# scrape ADP for all players
url_adp = "https://tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com/getNFLADP"
querystring = {"adpType":"PPR"}

headers = {
	"X-RapidAPI-Key": "9fe8d69ba4mshe735386b91fe9ffp10e56fjsn7c4b2b876e53",
	"X-RapidAPI-Host": "tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com"
}

response_adp = requests.get(url_adp, headers=headers, params=querystring)

print(response_adp.json())

{'statusCode': 200, 'body': {'adpDate': '20240601', 'adpType': 'PPR', 'adpList': [{'posADP': 'RB1', 'overallADP': '1.0', 'playerID': '3117251', 'longName': 'Christian McCaffrey'}, {'posADP': 'WR1', 'overallADP': '2.0', 'playerID': '4241389', 'longName': 'CeeDee Lamb'}, {'posADP': 'WR2', 'overallADP': '3.0', 'playerID': '3116406', 'longName': 'Tyreek Hill'}, {'posADP': 'WR3', 'overallADP': '4.0', 'playerID': '4362628', 'longName': "Ja'Marr Chase"}, {'posADP': 'WR4', 'overallADP': '5.5', 'playerID': '4262921', 'longName': 'Justin Jefferson'}, {'posADP': 'RB2', 'overallADP': '6.0', 'playerID': '4430807', 'longName': 'Bijan Robinson'}, {'posADP': 'WR5', 'overallADP': '7.0', 'playerID': '4374302', 'longName': 'Amon-Ra St. Brown'}, {'posADP': 'RB3', 'overallADP': '7.5', 'playerID': '4427366', 'longName': 'Breece Hall'}, {'posADP': 'WR6', 'overallADP': '9.0', 'playerID': '4426515', 'longName': 'Puka Nacua'}, {'posADP': 'WR7', 'overallADP': '10.0', 'playerID': '4047646', 'longName': 'A.J. Brow

In [49]:
adp_dict = response_adp.json()
adp_list = adp_dict['body']['adpList']
adp_df = pd.json_normalize(adp_list)
adp_df = adp_df.drop('teamAbv', axis=1)
adp_df = adp_df.drop('teamID', axis=1)
print(adp_df.head())
adp_df.to_csv('adp.csv', index=False)

  posADP overallADP playerID             longName
0    RB1        1.0  3117251  Christian McCaffrey
1    WR1        2.0  4241389          CeeDee Lamb
2    WR2        3.0  3116406          Tyreek Hill
3    WR3        4.0  4362628        Ja'Marr Chase
4    WR4        5.5  4262921     Justin Jefferson


In [3]:
# scrape projected stats for all weeks
url_projected = "https://tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com/getNFLProjections"

player_projected_stats_df = None
defense_projected_stats_df = None
NUM_WEEKS_IN_SEASON = 18

for week in range(1, NUM_WEEKS_IN_SEASON + 1):
    querystring_projected = {"week":f"{week}"}
    response_projected = requests.get(url_projected, headers=headers, params=querystring_projected)
    if response_projected is not None:
        projected_dict = response_projected.json()
        defense_projected_dict = projected_dict['body']['teamDefenseProjections']
        player_projected_dict = projected_dict['body']['playerProjections']
        defense_df = pd.DataFrame(defense_projected_dict.values())
        player_df = pd.json_normalize(player_projected_dict.values())
        player_df['week'] = week
        defense_df['week'] = week
        if player_projected_stats_df is None:
            player_projected_stats_df = player_df
        else:
            player_projected_stats_df = pd.concat([player_projected_stats_df, player_df], axis=0, ignore_index=True)
        if defense_projected_stats_df is None:
            defense_projected_stats_df = defense_df
        else:
            defense_projected_stats_df = pd.concat([defense_projected_stats_df, defense_df], axis=0, ignore_index=True)
        print(f'Done with week {week}.')
player_projected_stats_df.to_csv('original_player_projected_stats.csv', index=False)
defense_projected_stats_df.to_csv('original_defense_projected_stats.csv', index=False)


Done with week 1.
Done with week 2.
Done with week 3.
Done with week 4.
Done with week 5.
Done with week 6.
Done with week 7.
Done with week 8.
Done with week 9.
Done with week 10.
Done with week 11.
Done with week 12.
Done with week 13.
Done with week 14.
Done with week 15.
Done with week 16.
Done with week 17.
Done with week 18.


In [1]:
import nfl_data_py as nfl

In [5]:
weekly_datasets = nfl.import_weekly_data([2021, 2022, 2023], downcast=True)

Downcasting floats.


In [6]:
weekly_datasets = weekly_datasets.fillna('n/a')
print(weekly_datasets.head())
weekly_datasets.to_csv('original_weekly_player_stats(2021-2023).csv', index=False)

    player_id player_name player_display_name position position_group  \
0  00-0019596     T.Brady           Tom Brady       QB             QB   
1  00-0019596     T.Brady           Tom Brady       QB             QB   
2  00-0019596     T.Brady           Tom Brady       QB             QB   
3  00-0019596     T.Brady           Tom Brady       QB             QB   
4  00-0019596     T.Brady           Tom Brady       QB             QB   

                                        headshot_url recent_team  season  \
0  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
1  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
2  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
3  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
4  https://static.www.nfl.com/image/private/f_aut...          TB    2021   

   week season_type  ... receiving_first_downs  receiving_epa  \
0     1         REG  ...               

In [7]:
injuries = nfl.import_injuries([2021,2022,2023])
injuries = injuries.fillna('n/a')
print(injuries.head())
injuries.to_csv('original_injuries(2021-2023).csv', index=False)

   season game_type team  week     gsis_id position         full_name  \
0    2021       REG  ARI     1  00-0033258       TE   Darrell Daniels   
1    2021       REG  ARI     1  00-0034473       LB    Dennis Gardeck   
2    2021       REG  ARI     1  00-0035126       WR    Antoine Wesley   
3    2021       REG  ATL     1  00-0031583       DT     Grady Jarrett   
4    2021       REG  ATL     1  00-0030010       LB  Brandon Copeland   

  first_name last_name report_primary_injury report_secondary_injury  \
0    Darrell   Daniels                   n/a                     n/a   
1     Dennis   Gardeck                  Knee                    Hand   
2    Antoine    Wesley               Illness                     n/a   
3      Grady   Jarrett                   n/a                     n/a   
4    Brandon  Copeland                   n/a                     n/a   

  report_status               practice_primary_injury  \
0           n/a                                   Toe   
1           Ou

In [3]:
rosters = nfl.import_seasonal_rosters([2021, 2022, 2023])
rosters.to_csv('seasonal_rosters(2021-2023).csv')

In [8]:
weekly_rosters_2023 = nfl.import_weekly_rosters([2023])
weekly_rosters_2023.to_csv('weekly_rosters_2023.csv')
weekly_rosters_2022 = nfl.import_weekly_rosters([2022])
weekly_rosters_2022.to_csv('weekly_rosters_2022.csv')
weekly_rosters_2021 = nfl.import_weekly_rosters([2021])
weekly_rosters_2021.to_csv('weekly_rosters_2021.csv')


In [10]:
ftn_stats = nfl.import_ftn_data([2022, 2023], downcast=True, thread_requests=False)

Downcasting floats.


In [12]:
ftn_stats.to_csv('original_ftn_stats.csv', index=False)

In [2]:
win_totals = nfl.import_win_totals([2021, 2022, 2023])

In [3]:
win_totals.to_csv('win_totals.csv', index=False)